In [1]:

from unet import *

model = train_model(epochs=50, batch_size=8, learning_rate=0.5, num_workers=8, data_dir='./processed_data')

/media/zawiatgf/New Volume/Personal Files/Abdurrahman Zawia/University/Grad Project/Speech-To-Speech-Model/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda
Starting training... (LR Step Size: 7)
--- Starting Epoch 1/50 ---
Processing chunk_0_100...
Processing chunk_100_200...
Error during training on chunk chunk_100_200: CUDA out of memory. Tried to allocate 256.00 MiB. GPU 0 has a total capacity of 3.80 GiB of which 155.12 MiB is free. Including non-PyTorch memory, this process has 3.64 GiB memory in use. Of the allocated memory 3.25 GiB is allocated by PyTorch, and 310.84 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)
Processing chunk_200_300...
Error during training on chunk chunk_200_300: CUDA out of memory. Tried to allocate 128.00 MiB. GPU 0 has a total capacity of 3.80 GiB of which 25.12 MiB is free. Including non-PyTorch memory, this process has 3.77 GiB memory in use. Of the 

KeyboardInterrupt: 

In [ ]:
model.eval()


UNetSpectrogramTranslator(
  (enc1): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (enc2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (enc3): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (bottleneck): Sequential(
    (0): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (up3): ConvTranspose2d(512, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), ou

In [ ]:
from unet import *
from dataset_loader import *

model = UNetSpectrogramTranslator()
model.load("Unet_model_case2")
model.eval()
data_dict = dataset_loader.load_data(start_idx=0, num_samples=1, split="test", lang="en")
play_audio(data_dict['en'][0])


/media/zawiatgf/New Volume/Personal Files/Abdurrahman Zawia/University/Grad Project/Speech-To-Speech-Model/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Model loaded from Unet_model_case2
Buffering data into memory...
Buffered 1 samples (0.00 MB). Converting to Dataset...


In [ ]:


def resize_spectrogram(spec, target_size=(256, 256)):
    """Resizes spectrogram to match model input size."""
    # spec shape: (freq, time) or (1, freq, time)
    if isinstance(spec, list):
        spec = np.array(spec)
    
    t_spec = torch.tensor(spec, dtype=torch.float32)
    
    # Ensure (Batch, Channel, H, W) -> (1, 1, F, T)
    if t_spec.ndim == 2:
        t_spec = t_spec.unsqueeze(0).unsqueeze(0)
    elif t_spec.ndim == 3:
        t_spec = t_spec.unsqueeze(0)
        
    t_resized = F.interpolate(t_spec, size=target_size, mode='bilinear', align_corners=False)
    return t_resized


sample = data_dict['en'][0]
print(f"Sample ID: {sample['id']}")

# Encode
print("Encoding audio...")
encoder = SpectogramEncoder()
raw_audio = np.array(sample['audio']['array'])
spectrogram = encoder.encode(raw_audio, sample['audio']['sampling_rate'])
print(spectrogram.shape)

print("Resizing spectrogram...")
input_tensor = resize_spectrogram(spectrogram)

print("Running inference...")
with torch.no_grad():
    output_tensor = model(input_tensor)

# Decode
print("Decoding output...")
output_spec = output_tensor.squeeze().cpu().numpy()

reconstructed_audio = encoder.decode(output_spec, sample['audio']['sampling_rate'])
rec_audio = Audio(data=reconstructed_audio, rate=sample['audio']['sampling_rate'])
rec_audio

Sample ID: 1904
Encoding audio...
(1025, 331)
Resizing spectrogram...
Running inference...
Decoding output...
